<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -q
!wandb login

     |████████████████████████████████| 1.9 MB 15.4 MB/s 
     |████████████████████████████████| 166 kB 63.2 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 67.2 MB/s 
     |████████████████████████████████| 162 kB 76.8 MB/s 
     |████████████████████████████████| 162 kB 74.0 MB/s 
     |████████████████████████████████| 158 kB 68.2 MB/s 
     |████████████████████████████████| 157 kB 76.5 MB/s 
     |████████████████████████████████| 157 kB 78.0 MB/s 
     |████████████████████████████████| 157 kB 77.5 MB/s 
     |████████████████████████████████| 157 kB 14.3 MB/s 
     |████████████████████████████████| 157 kB 93.7 MB/s 
     |████████████████████████████████| 157 kB 60.9 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 156 kB 64.8 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))

80134624/80134624 [==============================] - 3s 0us/step


In [6]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [7]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
#class_names == test_class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [9]:
model_vgg19 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg19,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
vgg19.trainable=False
model_vgg19.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 32)                802848    
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 20,827,397
Trainable params: 803,013
Non

In [10]:
model_vgg19.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [11]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "vgg19",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [12]:
%%time
epochs=40
history = model_vgg19.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.2756 - accuracy: 0.5526

wandb: Adding directory to artifact (/content/wandb/run-20221104_125916-1d53iqcr/files/model-best)... Done. 1.0s


22/22 [==============================] - 232s 8s/step - loss: 1.2756 - accuracy: 0.5526 - val_loss: 0.7098 - val_accuracy: 0.6857
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.6957 - accuracy: 0.7273

wandb: Adding directory to artifact (/content/wandb/run-20221104_125916-1d53iqcr/files/model-best)... Done. 0.7s


...

22/22 [==============================] - 14s 605ms/step - loss: 0.1252 - accuracy: 0.9489 - val_loss: 0.1246 - val_accuracy: 0.9486
Epoch 19/40
22/22 [==============================] - 10s 422ms/step - loss: 0.1027 - accuracy: 0.9560 - val_loss: 0.1709 - val_accuracy: 0.9429
Epoch 20/40
22/22 [==============================] - 10s 426ms/step - loss: 0.0969 - accuracy: 0.9588 - val_loss: 0.1460 - val_accuracy: 0.9429
Epoch 21/40
22/22 [==============================] - 10s 431ms/step - loss: 0.0887 - accuracy: 0.9631 - val_loss: 0.1933 - val_accuracy: 0.9429
Epoch 22/40
22/22 [==============================] - 10s 429ms/step - loss: 0.0944 - accuracy: 0.9560 - val_loss: 0.1931 - val_accuracy: 0.9371
Epoch 23/40
22/22 [==============================] - 10s 425ms/step - loss: 0.1049 - accuracy: 0.9631 - val_loss: 0.1672 - val_accuracy: 0.9429
Epoch 24/40
22/22 [==============================] - 10s 420ms/step - loss: 0.0854 - accuracy: 0.9616 - val_loss: 0.1377 - val_accuracy: 0.9486
Epoc

wandb: Adding directory to artifact (/content/wandb/run-20221104_125916-1d53iqcr/files/model-best)... Done. 0.9s


22/22 [==============================] - 14s 624ms/step - loss: 0.0702 - accuracy: 0.9688 - val_loss: 0.1205 - val_accuracy: 0.9600
Epoch 28/40
22/22 [==============================] - 10s 430ms/step - loss: 0.0808 - accuracy: 0.9673 - val_loss: 0.1533 - val_accuracy: 0.9314
Epoch 29/40
22/22 [==============================] - 10s 423ms/step - loss: 0.0883 - accuracy: 0.9645 - val_loss: 0.1493 - val_accuracy: 0.9314
Epoch 30/40
22/22 [==============================] - 10s 432ms/step - loss: 0.0704 - accuracy: 0.9759 - val_loss: 0.1227 - val_accuracy: 0.9543
Epoch 31/40
22/22 [==============================] - 10s 430ms/step - loss: 0.0675 - accuracy: 0.9815 - val_loss: 0.1670 - val_accuracy: 0.9429
Epoch 32/40
22/22 [==============================] - 10s 424ms/step - loss: 0.0902 - accuracy: 0.9616 - val_loss: 0.1670 - val_accuracy: 0.9486
Epoch 33/40
22/22 [==============================] - 10s 423ms/step - loss: 0.0868 - accuracy: 0.9602 - val_loss: 0.1418 - val_accuracy: 0.9429
Epoc

wandb: Adding directory to artifact (/content/wandb/run-20221104_125916-1d53iqcr/files/model-best)... Done. 0.9s


22/22 [==============================] - 14s 611ms/step - loss: 0.0764 - accuracy: 0.9702 - val_loss: 0.1190 - val_accuracy: 0.9657
Epoch 37/40
22/22 [==============================] - 10s 419ms/step - loss: 0.0821 - accuracy: 0.9744 - val_loss: 0.1376 - val_accuracy: 0.9543
Epoch 38/40
22/22 [==============================] - 10s 431ms/step - loss: 0.0756 - accuracy: 0.9759 - val_loss: 0.1521 - val_accuracy: 0.9429
Epoch 39/40
22/22 [==============================] - 10s 429ms/step - loss: 0.0773 - accuracy: 0.9787 - val_loss: 0.1453 - val_accuracy: 0.9486
Epoch 40/40
22/22 [==============================] - 10s 431ms/step - loss: 0.0700 - accuracy: 0.9815 - val_loss: 0.1236 - val_accuracy: 0.9600
CPU times: user 2min 21s, sys: 37.9 s, total: 2min 59s
Wall time: 12min 14s


In [13]:
wandb.finish()

accuracy,▁▄▅▆▆▇▇▇▇▇▇▇▇████▇██████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▆▇▇▇▇▇█▇▇██▇███▇▇▇▇▇████▇▇█▇█▇████▇██
val_loss,█▇▅▄▃▃▃▂▂▂▂▁▁▂▂▁▁▁▂▁▂▂▂▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁
accuracy,0.98153
best_epoch,35
best_val_loss,0.11899
epoch,39
loss,0.07
val_accuracy,0.96


In [14]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
